In [1]:
from ultralytics import YOLO
import cv2
import pandas as pd
import numpy as np
import time
import os
from datetime import datetime

model = YOLO('runs/best version/v_8/YOLO_V8.pt')

purchase_history = pd.DataFrame(columns=['Datetime', 'Items', 'Total Cost', 'Process_Time'])

# Global price list variable and DataFrame to hold process information
df = pd.read_excel("price_list.xlsx")
price_list = dict(zip(df['food_name'], df['price']))
process_log = pd.DataFrame(columns=['Datetime', 'Items', 'Total Cost', 'Process_Time'])

# Detect food in the image and save the predicted image
def detect_food(image_path, save_path):
    results = model(image_path)
    boxes = results[0].boxes  # Get the first result
    data = boxes.xyxy.cpu().numpy()  # x1, y1, x2, y2
    confidences = boxes.conf.cpu().numpy()  # confidence values
    classes = boxes.cls.cpu().numpy()  # class ids

    # Create a DataFrame by combining columns
    predictions = pd.DataFrame(data, columns=['x1', 'y1', 'x2', 'y2'])
    predictions['confidence'] = confidences
    predictions['class'] = classes
    predictions['name'] = predictions['class'].apply(lambda x: model.names[int(x)])
    
    # Save the predicted image
    img = cv2.imread(image_path)
    for i, row in predictions.iterrows():
        x1, y1, x2, y2 = int(row['x1']), int(row['y1']), int(row['x2']), int(row['y2'])
        label = f"{row['name']} {row['confidence']:.2f}"
        img = cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
        img = cv2.putText(img, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
    cv2.imwrite(save_path, img)
    print(f"Predicted image saved: {save_path}")
    
    return predictions

# Count the foods and their quantities
def count_foods(predictions):
    food_counts = predictions['name'].value_counts().to_dict()
    return food_counts


def calculate_individual_costs(food_counts):
    global price_list
    individual_costs = {}
    for food, count in food_counts.items():
        if food in price_list:
            individual_costs[food] = price_list[food] * count
    return individual_costs


def calculate_total_cost(individual_costs):
    return sum(individual_costs.values())

# Capture an image from the camera
def capture_image(camera_index, save_path):
    cap = cv2.VideoCapture(camera_index)
    if not cap.isOpened():
        print(f"Failed to open camera {camera_index}")
        return False
    ret, frame = cap.read()
    if ret:
        cv2.imwrite(save_path, frame)
        print(f"Image captured and saved: {save_path}")
    cap.release()
    return ret

# Main function
def main(image_path, prediction_image_path):
    global process_log
    
    # Start time to measure process duration
    start_time = time.time()
    
    predictions = detect_food(image_path, prediction_image_path)
    food_counts = count_foods(predictions)
    
    individual_costs = calculate_individual_costs(food_counts)
    total_cost = calculate_total_cost(individual_costs)
        
    # Add process duration and other information to DataFrame
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    items_string = ', '.join([f"{food}: {count}" for food, count in food_counts.items()])
    new_log = pd.DataFrame([{
        'Datetime': current_time,
        'Items': items_string,
        'Total Cost': total_cost
    }])
    process_log = pd.concat([process_log, new_log], ignore_index=True)
    
    # Print the results
    print("Detected food items and counts:")
    for food, count in food_counts.items():
        print(f"{food}: {count} items, Total Cost: {individual_costs.get(food, 0)} TL")

    print(f"\nTotal Cost: {total_cost} TL")
    end_time = time.time()
    duration = end_time - start_time
    return food_counts, total_cost, duration

In [2]:
# Usage
history_dir = 'history_images'
prediction_dir = 'history_images_prediction'
os.makedirs(history_dir, exist_ok=True)
os.makedirs(prediction_dir, exist_ok=True)

# External camera index
camera_index = 1

# Capture an image and save it
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
image_filename = f"{history_dir}/image_{current_time}.jpg"
prediction_image_filename = f"{prediction_dir}/prediction_{current_time}.jpg"

# Start time to measure process duration
start_time = time.time()

if capture_image(camera_index, image_filename):
    food_counts, total_cost, process_duration = main(image_filename, prediction_image_filename)
else:
    print("Failed to capture an image")

# End time to measure process duration
end_time = time.time()
overall_duration = round(end_time - start_time, 1)

print(f"\nOverall process duration: {overall_duration} seconds")

# Add process duration to DataFrame
process_log.loc[process_log.index[-1], 'Process_Time'] = overall_duration
purchase_history = pd.concat([purchase_history, process_log], ignore_index=True)
process_log = []
process_log = pd.DataFrame(columns=['Datetime', 'Items', 'Total Cost', 'Process_Time'])

Image captured and saved: history_images/image_20240610_122913.jpg

image 1/1 d:\Belgeler\2023-2024\bitirme 2\history_images\image_20240610_122913.jpg: 608x800 1 FP Pilav, 1 Mineral water, 422.7ms
Speed: 2.1ms preprocess, 422.7ms inference, 910.2ms postprocess per image at shape (1, 3, 608, 800)
Predicted image saved: history_images_prediction/prediction_20240610_122913.jpg
Detected food items and counts:
Mineral water: 1 items, Total Cost: 22 TL
FP Pilav: 1 items, Total Cost: 40 TL

Total Cost: 62 TL

Overall process duration: 6.3 seconds


In [3]:
# İstediğiniz zaman işlem bilgilerini kaydetme
current_date = datetime.now().strftime("%d-%m-%Y")
filename = f"purchase_history_{current_date}.xlsx"
purchase_history.to_excel(filename, index=False)